<a href="https://colab.research.google.com/github/Satvik256/Machine-Learning/blob/main/Grocery_Sale_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print (" Installing FastAI libraries ... (takes 2 min)")
!pip install fastai==0.7.0 > /dev/null
!pip install scikit-learn==0.20 > null
print ("\n Clonning FastAI repository locally ...")
!git clone https://github.com/fastai/fastai.git fastai_ml
!ln -s fastai_ml/courses/ml1/fastai/ fastai

 Installing FastAI libraries ... (takes 2 min)
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 0.3.1 which is incompatible.

 Clonning FastAI repository locally ...
Cloning into 'fastai_ml'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 11993 (delta 66), reused 42 (delta 22), pack-reused 11864
Receiving objects: 100% (11993/11993), 565.40 MiB | 34.21 MiB/s, done.
Resolving deltas: 100% (9392/9392), done.


In [ ]:
print ("\n Installing required libraries...")
!pip install --upgrade setuptools > /dev/null
!pip install feather > /dev/null
!pip install scikit-misc==0.1.0 > /dev/null
!pip install pdpbox==0.2.0 > /dev/null
!pip install treeinterpreter==0.2.2 > /dev/null
print ("\n Downloading datasets...")
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-train.7z
!7z x -y grocery-sales-train.7z > /dev/null
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-test.tar.gz
!tar xvf grocery-sales-test.tar.gz > /dev/null
print ("\n Importing libraries")
import pandas as pd
import os
import numpy as np


 Installing required libraries...
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.

--2020-10-09 21:26:13--  https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-train.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23818196 (23M) [application/octet-stream]
Saving to: ‘grocery-sales-train.7z’

grocery-sales-train 100%[===================>]  22.71M  63.4MB/s    in 0.4s    

2020-10-09 21:26:14 (63.4 MB/s) - ‘grocery-sales-train.7z’ saved [23818196/23818196]

--2020-10-09 21:26:16--  https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-test.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [ ]:
PATH = 'grocery-sales/'
!ls {PATH}

test.csv  train_basic.csv


In [ ]:
types = {'id': 'int64',
        'item_nbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': 'object'}

In [ ]:
%%time
df_all = pd.read_csv(f'{PATH}train_basic.csv', parse_dates=['date'], dtype=types, 
                     infer_datetime_format=True)#, skiprows=range(1,100000000))

CPU times: user 1.47 s, sys: 115 ms, total: 1.59 s
Wall time: 1.6 s


In [ ]:
df_all.onpromotion.fillna(False, inplace=True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

os.makedirs('tmp', exist_ok=True)
%time df_all.to_feather('tmp/raw_groceries')

CPU times: user 89.4 ms, sys: 61.2 ms, total: 151 ms
Wall time: 1.43 s


In [ ]:
df_all.drop('Unnamed: 0', axis=1, inplace=True)
%time df_all.describe(include='all')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


CPU times: user 338 ms, sys: 2.26 ms, total: 340 ms
Wall time: 354 ms


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.997040e+06,1997040,1.997040e+06,1.997040e+06,1.997040e+06,1997040
unique,NaN,1684,NaN,NaN,NaN,2
top,NaN,2017-05-07 00:00:00,NaN,NaN,NaN,False
freq,NaN,1908,NaN,NaN,NaN,1528448
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,6.271723e+07,NaN,2.745692e+01,9.728621e+05,8.551911e+00,NaN
std,3.619772e+07,NaN,1.633284e+01,5.202940e+05,2.057796e+01,NaN
min,2.060000e+02,NaN,1.000000e+00,9.699500e+04,-1.768000e+03,NaN
25%,3.138601e+07,NaN,1.200000e+01,5.227210e+05,2.000000e+00,NaN


In [ ]:
df_test = pd.read_csv(f'{PATH}test.csv',parse_dates=['date'], dtype=types, infer_datetime_format=True)

df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_test.onpromotion = df_all.onpromotion.astype(bool)
df_test.describe(include='all')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,1997040
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-27 00:00:00,NaN,NaN,False
freq,NaN,210654,NaN,NaN,1528448
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


In [ ]:
df_all.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
1997035,73734366,2016-03-18,54,1464238,22.0,False
1997036,101679247,2016-12-31,49,1489899,29.0,True
1997037,124420242,2017-08-05,38,849080,2.0,False
1997038,35980557,2014-11-25,31,1463798,3.0,False
1997039,85369289,2016-07-18,28,362035,14.0,False


In [ ]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [ ]:
%time add_datepart(df_all, 'date')

/usr/local/lib/python3.6/dist-packages/fastai/structured.py:114: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())


CPU times: user 2.87 s, sys: 208 ms, total: 3.08 s
Wall time: 3.09 s


In [ ]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [ ]:
n_valid = len(df_test)
n_trn = len(df_all) - n_valid
train, valid = split_vals(df_all, n_trn)
train.shape, valid.shape

((623616, 18), (1373424, 18))

In [ ]:
# train_cats(raw_train)
# apply_cats(raw_valid, raw_train)

In [ ]:
%%time
trn, y, nas = proc_df(train, 'unit_sales')
val, y_val, nas = proc_df(valid, 'unit_sales')

CPU times: user 362 ms, sys: 273 ms, total: 635 ms
Wall time: 640 ms


In [ ]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val),
          m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
set_rf_samples(1000000)

In [ ]:
%time x = np.array(trn, dtype=np.float32)

CPU times: user 526 ms, sys: 42.8 ms, total: 569 ms
Wall time: 570 ms


In [ ]:
# set n_jobs=-1 to use all CPU cores available
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=100, n_jobs=-1) 
%time m.fit(x, y)

CPU times: user 2min 37s, sys: 93.6 ms, total: 2min 37s
Wall time: 1min 21s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=100, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
print_score(m)

[0.7749968858159555, 0.7914586463576803, 0.22919178846958188, 0.19325335658455156]


In [ ]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=10, n_jobs=-1)
%time m.fit(x, y)

CPU times: user 3min 18s, sys: 139 ms, total: 3min 18s
Wall time: 1min 41s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
print_score(m)

[0.6088583814540425, 0.7606842417474565, 0.5242497824508892, 0.25477133706824606]


In [ ]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, n_jobs=-1)
%time m.fit(x, y)

CPU times: user 3min 44s, sys: 284 ms, total: 3min 44s
Wall time: 1min 54s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
%time print_score(m)

[0.39528156732384073, 0.7705696284129441, 0.7994791468831824, 0.2352764119231353]
CPU times: user 1min 1s, sys: 350 ms, total: 1min 1s
Wall time: 33.4 s
